In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats
eps = np.finfo(float).eps

In [2]:
class BayesLinReg(object):
	"""
	Bayesian linear regression.
	See: Pattern Recognition and Machine Learning by Christopher Bishop ch.3.
		https://www.microsoft.com/en-us/research/people/cmbishop/prml-book/
	Blogs: https://maxhalford.github.io/blog/bayesian-linear-regression/
	"""
	def __init__(self, num_feas, alpha, beta):
		self.num_feas = num_feas
		self.alpha = alpha
		self.beta = beta
		self.mean = np.zeros((num_feas,1))
		self.invcov_mat = np.identity(num_feas) / alpha
		return

	def update(self, x, y):
		"""
		eq 3.50-3.51 in Bishop
		"""
		invcov_mat_n = self.invcov_mat + self.beta * np.outer(x, x)
		mean_n = np.matmul(np.linalg.inv(invcov_mat_n), (np.matmul(self.invcov_mat, self.mean) + self.beta* np.expand_dims(np.dot(y, x), axis=1)))
		assert mean_n.shape == self.mean.shape
		self.mean = mean_n
		self.invcov_mat = invcov_mat_n
		return self

	def predict(self, x):
		"""
		eq 3.58-3.59 in Bishop
		"""
		pred_mean = np.dot(x, self.mean)
		sigma_squared_x = 1./self.beta + np.dot(np.dot(x, np.linalg.inv(self.invcov_mat)), x.T)
		return stats.norm(loc=pred_mean.T, scale=sigma_squared_x ** .5)

	@property
	def weights_dist(self):
		return stats.multivariate_normal(mean=self.mean, cov=np.linalg.inv(self.invcov_mat))

In [3]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [4]:
ni, nd = X.shape

In [5]:
alpha=0.3
beta=1

In [6]:
model = BayesLinReg(num_feas=nd, alpha=alpha, beta=beta)

In [7]:
X[0,:,np.newaxis].shape

(10, 1)

In [8]:
res = model.predict(X[0])

In [9]:
res.mean()

array([0.])

In [10]:
res.var()

array([1.00422081])

In [11]:
model.mean.shape

(10, 1)

In [24]:
ures = model.update(X[0], y[0])

In [25]:
model.mean

array([[ 5.10981307],
       [ 6.80130711],
       [ 8.27967374],
       [ 2.93528522],
       [-5.93482418],
       [-4.67297054],
       [-5.82442361],
       [-0.34788336],
       [ 2.6717239 ],
       [-2.36812224]])

In [14]:
a = np.expand_dims(np.dot(y[0], X[0]), axis=1)

In [15]:
a.shape

(10, 1)

In [16]:
b = np.matmul(model.invcov_mat, model.mean)

In [17]:
b.shape

(10, 1)

In [18]:
c = a + b

In [19]:
c.shape

(10, 1)

In [20]:
invcov_mat_n = model.invcov_mat + model.beta * np.outer(X[0], X[0])

In [21]:
invcov_mat_n.shape

(10, 10)